#### Setup access token

In [1]:
import pandas as pd
import webbrowser
# Import the required module from the fyers_apiv3 package
from fyers_apiv3 import fyersModel
from datetime import datetime, timedelta
import pandas as pd
import time

# Configuration

In [2]:
def generate_fyers_access_token(
    client_id: str,
    secret_key: str,
    redirect_uri: str,
    state: str = "sample_state"
) -> dict:
    """
    Handles FYERS OAuth flow and returns access token response.

    Returns
    -------
    dict
        Full response containing access_token, refresh_token, expiry, etc.
    """

    # -------------------------------
    # Step 1: Generate auth URL
    # -------------------------------
    session = fyersModel.SessionModel(
        client_id=client_id,
        secret_key=secret_key,
        redirect_uri=redirect_uri,
        response_type="code",
        state=state
    )

    auth_url = session.generate_authcode()
    webbrowser.open(auth_url)

    # -------------------------------
    # Step 2: User pastes redirected URL
    # -------------------------------
    raw_code = input("Paste redirected URL here: ").strip()

    try:
        auth_code = raw_code.split("auth_code=")[1].split("&")[0]
    except IndexError:
        raise ValueError("Invalid redirect URL. Auth code not found.")

    # -------------------------------
    # Step 3: Exchange auth_code for access_token
    # -------------------------------
    session = fyersModel.SessionModel(
        client_id=client_id,
        secret_key=secret_key,
        redirect_uri=redirect_uri,
        response_type="code",
        grant_type="authorization_code"
    )

    session.set_token(auth_code)
    token_response = session.generate_token()

    return token_response


In [3]:
client_id="ALT6RUE1IF-100"
secret_key="0UT0LW5PE4"
redirect_uri="https://luvratan.tech/"

token_response = generate_fyers_access_token(
    client_id=client_id,
    secret_key=secret_key,
    redirect_uri=redirect_uri
)

access_token = token_response["access_token"]
print("Access Token:", access_token)


Access Token: eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJhdWQiOlsiZDoxIiwiZDoyIiwieDowIiwieDoxIl0sImF0X2hhc2giOiJnQUFBQUFCcGdIU2dMc2t3XzhLUWZCS1R1VWFXSEZsWE1DV0tkaUNlOUYtd3Jwd2tORk1JQ3BsWG5yTkVCcGJwbktKNy0tVGxRbGEyRXAxd2VkbkJmLVdYR0R6OFkxX19JVlVSZk5tMUxTVmpMbGNWWVFQb1JvYz0iLCJkaXNwbGF5X25hbWUiOiIiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiJiMjM2ZDk2Mjg3YzdkYjI0NzdmOTE0Y2FhZWQ5Y2YwZTNmYTAxYzQxMzgzMWIxMmMxYjRhYjlhYyIsImlzRGRwaUVuYWJsZWQiOiJOIiwiaXNNdGZFbmFibGVkIjoiTiIsImZ5X2lkIjoiWEwwMDcyMSIsImFwcFR5cGUiOjEwMCwiZXhwIjoxNzcwMDc4NjAwLCJpYXQiOjE3NzAwMjYxNDQsImlzcyI6ImFwaS5meWVycy5pbiIsIm5iZiI6MTc3MDAyNjE0NCwic3ViIjoiYWNjZXNzX3Rva2VuIn0.JO3WOwcpRlSvGpsE11NGEC8SMeir4nwjKuKBqVh5yqw


#### Fetch Data

In [4]:
import time

resolution = 5
bars_needed = 203

# Request 10 days of data to ensure we have enough bars
days_to_fetch = 5
lookback_seconds = days_to_fetch * 24 * 60 * 60  # 10 days in seconds

# Avoid partial candles as per API documentation
range_to = int(time.time())
range_from = range_to - lookback_seconds

print(f"Fetching {days_to_fetch} days of {resolution}-minute data")

Fetching 5 days of 5-minute data


In [5]:

# Initialize the FyersModel instance with your client_id, access_token, and enable async mode
fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path="")

data = {
    "symbol": "NSE:ADANIPORTS-EQ",
    "resolution": resolution,
    "date_format": "0",
    "range_from": range_from,
    "range_to": range_to,
    "cont_flag": "1"
}

response = fyers.history(data=data)
print(response)

{'candles': [[1769658300, 1384.9, 1384.9, 1371.6, 1375, 52370], [1769658600, 1375, 1377.1, 1373.5, 1375.4, 17526], [1769658900, 1375.7, 1379.6, 1375.1, 1376.7, 20145], [1769659200, 1376.6, 1377.5, 1375, 1377.5, 11279], [1769659500, 1377.2, 1379.5, 1376.9, 1379, 30793], [1769659800, 1379, 1382, 1378.5, 1378.5, 24159], [1769660100, 1378.5, 1382.5, 1378.5, 1382.2, 15014], [1769660400, 1383.5, 1387, 1381.7, 1381.9, 98417], [1769660700, 1381.5, 1387.7, 1381.2, 1385.4, 62747], [1769661000, 1386, 1386.4, 1384, 1385.4, 12004], [1769661300, 1385.1, 1386, 1382.1, 1382.7, 129874], [1769661600, 1382.5, 1382.5, 1378.6, 1378.7, 19766], [1769661900, 1378.9, 1380.7, 1377, 1377.6, 22139], [1769662200, 1377.6, 1378.3, 1373.1, 1377.5, 28370], [1769662500, 1377.3, 1378.2, 1374.9, 1375.2, 8968], [1769662800, 1375.3, 1378.9, 1374.1, 1378.7, 5221], [1769663100, 1378.4, 1380, 1376.7, 1379, 12954], [1769663400, 1378.3, 1383.4, 1377.3, 1380, 17911], [1769663700, 1379.8, 1380.9, 1377.1, 1379, 8328], [1769664000,

In [6]:
def fyers_history_to_df(response: dict) -> pd.DataFrame:
    """
    Convert FYERS history API response to a pandas DataFrame.
    Shifts intraday candle timestamps forward by 15 minutes.
    """

    if response.get("s") != "ok":
        raise ValueError(f"FYERS API error: {response}")
    print(response["candles"])
    df = pd.DataFrame(
        response["candles"],
        columns=["timestamp", "open", "high", "low", "close", "volume"]
    )

    # Convert UNIX epoch → timezone-aware datetime (IST)
    df["datetime"] = (
        pd.to_datetime(df["timestamp"], unit="s", utc=True)
        .dt.tz_convert("Asia/Kolkata")
    )

    # Shift candle time forward by n minutes
    # df["datetime"] = df["datetime"] + pd.Timedelta(minutes=resolution)

    # Sort by adjusted time
    df = df.sort_values("datetime")

    # Drop raw timestamp if not needed
    df = df.drop(columns=["timestamp"])
    df = df.set_index("datetime")

    return df


df = fyers_history_to_df(response)


[[1769658300, 1384.9, 1384.9, 1371.6, 1375, 52370], [1769658600, 1375, 1377.1, 1373.5, 1375.4, 17526], [1769658900, 1375.7, 1379.6, 1375.1, 1376.7, 20145], [1769659200, 1376.6, 1377.5, 1375, 1377.5, 11279], [1769659500, 1377.2, 1379.5, 1376.9, 1379, 30793], [1769659800, 1379, 1382, 1378.5, 1378.5, 24159], [1769660100, 1378.5, 1382.5, 1378.5, 1382.2, 15014], [1769660400, 1383.5, 1387, 1381.7, 1381.9, 98417], [1769660700, 1381.5, 1387.7, 1381.2, 1385.4, 62747], [1769661000, 1386, 1386.4, 1384, 1385.4, 12004], [1769661300, 1385.1, 1386, 1382.1, 1382.7, 129874], [1769661600, 1382.5, 1382.5, 1378.6, 1378.7, 19766], [1769661900, 1378.9, 1380.7, 1377, 1377.6, 22139], [1769662200, 1377.6, 1378.3, 1373.1, 1377.5, 28370], [1769662500, 1377.3, 1378.2, 1374.9, 1375.2, 8968], [1769662800, 1375.3, 1378.9, 1374.1, 1378.7, 5221], [1769663100, 1378.4, 1380, 1376.7, 1379, 12954], [1769663400, 1378.3, 1383.4, 1377.3, 1380, 17911], [1769663700, 1379.8, 1380.9, 1377.1, 1379, 8328], [1769664000, 1378.9, 138

In [7]:
df.tail()

,open,high,low,close,volume
datetime,,,,,
2026-02-02 15:05:00+05:30,1401.1,1405.0,1400.4,1404.2,42048
2026-02-02 15:10:00+05:30,1403.5,1406.0,1403.5,1404.7,47392
2026-02-02 15:15:00+05:30,1404.8,1405.8,1402.5,1404.5,70981
2026-02-02 15:20:00+05:30,1404.1,1404.2,1401.0,1401.6,61264
2026-02-02 15:25:00+05:30,1400.9,1401.4,1400.1,1400.5,5767


In [8]:
df.to_csv("demo_data.csv", index="datetime")